In [117]:
from openai import OpenAI
from dotenv import  load_dotenv , find_dotenv
import os

_ : bool = load_dotenv(find_dotenv())

client : OpenAI = OpenAI()

In [118]:
import json
def get_student_data(student_name) :
     """Returns the student data for the given student name."""

     details = [
          {
              'name':' smith',
              'age':20,
              'gender':'male'
          },
          {
              'name':'lisa',
              'age':22,
              'gender':'female'
          },
          {
              'name':'janes',
              'age':23,
              'gender':'male'
          },
          {
              'name':'james',
              'age':24,
              'gender':'male'
          },
          {
              'name':'jessica',
              'age':19,
              'gender':'female'
          },
     ]

     for i in details:
          if i['name'].lower() == student_name.lower():
                return json.dumps(i)

     



def get_teacher_data(teacher_name):
     """Returns the teacher details for the given teacher name."""
     details = [
          { 
               "name": 'Paul',
               'age':35,
               'subject':'chemistry',
          },
          { 
               "name": 'Mary',
               'age':30,
              'subject':'english',
          },
          { 
               "name": 'John',
               'age':25,
             'subject':'maths',
          },
          { 
               "name": 'Jane',
               'age':32,
             'subject':'Biology',
          },
     ]

     for i in details:
           if i['name'].lower() == teacher_name.lower():
                return json.dumps(i)




In [119]:
available_functions = {
              "get_student_data" : get_student_data,
              "get_teacher_data" : get_teacher_data 
             }

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_student_data",
            "description": "Returns the student data for the given student name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "student_name": {
                        "type": "string",
                        "description": "The name of the student."
                    }
                },
                "required": ["student_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_teacher_data",
            "description": "Returns the teacher data for the given teacher name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "teacher_name": {
                        "type": "string",
                        "description": "The name of the teacher."
                    }
                },
                "required": ["teacher_name"]
            }
        }
    }
]



messages = [
    {
        "role": "user",
        "content": "Do You have any information about teacher John and student lisa?"
    }
]

In [120]:

# creating the first chat completion

response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

response_message = response.choices[0].message
dict(response_message)

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_xzgryAzeUnqJC04D1xJBoy5J', function=Function(arguments='{"teacher_name": "John"}', name='get_teacher_data'), type='function'),
  ChatCompletionMessageToolCall(id='call_rsxp6qVxiyS3lp8ylXUxBcNe', function=Function(arguments='{"student_name": "Lisa"}', name='get_student_data'), type='function')]}

In [121]:
tool_calls = response_message.tool_calls
list(tool_calls)

[ChatCompletionMessageToolCall(id='call_xzgryAzeUnqJC04D1xJBoy5J', function=Function(arguments='{"teacher_name": "John"}', name='get_teacher_data'), type='function'),
 ChatCompletionMessageToolCall(id='call_rsxp6qVxiyS3lp8ylXUxBcNe', function=Function(arguments='{"student_name": "Lisa"}', name='get_student_data'), type='function')]

In [122]:
if tool_calls: 
     messages.append(response_message)

     for tool_call in tool_calls:
            function_name = tool_call.function.name   
            print(function_name)         
            function_to_call = available_functions[function_name]
            print(function_to_call)
            args_data = json.loads(tool_call.function.arguments)
            print(args_data)

            if 'student_name' in args_data:
                  args_to_pass = args_data['student_name']
            elif 'teacher_name' in args_data:
                  args_to_pass =   args_data['teacher_name']
                  
            function_response = function_to_call(
             args_to_pass
            )
 
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
             )  
           
            print(function_response)



get_teacher_data
<function get_teacher_data at 0x00000220A50CFB00>
{'teacher_name': 'John'}
{"name": "John", "age": 25, "subject": "maths"}
get_student_data
<function get_student_data at 0x00000220A50CFE20>
{'student_name': 'Lisa'}
{"name": "lisa", "age": 22, "gender": "female"}


In [123]:
second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function

In [124]:
second_response.choices[0].message.content

'Yes, Teacher John is 25 years old and teaches math. Student Lisa is 22 years old and is female. Is there anything specific you would like to know about them?'